<h1>Databases II</h1>

<p>After working with SQLite, we will now move to a more advanced database system - PostgreSQL. Unlike SQLite, this database has a server application and does not exist in a file or RAM.</p>

<p>Install PostgreSQL as directed</p>

<h2>PIP package manager</h2>

<p>One of the most popular and common package managers for Python is PIP. A package manager allows a programmer to install 3rd-party modules into Python, extending the functionality available to programs. We will use PIP to install a package that Python can use to connect to a PostgreSQL server, as this functionality is not available by default.
From your terminal run:<br/>
    <b>pip install psycopg2</b>
</p>

<h2>Create a Database</h2>

In [1]:
# get the relevant module
import psycopg2

In [2]:
# queries to create the relevant tables
createAccountsQry = '''
CREATE SEQUENCE IF NOT EXISTS public.vb_accounts_acc_id_seq
    INCREMENT 1
    START 1
    MINVALUE 1
    MAXVALUE 2147483647
    CACHE 1;

ALTER SEQUENCE public.vb_accounts_acc_id_seq
    OWNER TO gini;
    
CREATE TABLE IF NOT EXISTS public.vb_accounts
(
    acc_id integer NOT NULL DEFAULT nextval('public.vb_accounts_acc_id_seq'::regclass),
    acc_ctime timestamp with time zone NOT NULL DEFAULT now(),
    acc_name character varying(100) COLLATE pg_catalog."default" NOT NULL,
    acc_nat_id character varying(8) COLLATE pg_catalog."default" NOT NULL,
    acc_code character varying(8) COLLATE pg_catalog."default" NOT NULL,
    acc_age integer NOT NULL,
    acc_amount double precision NOT NULL DEFAULT 0.0,
    acc_pass character varying(100) COLLATE pg_catalog."default" NOT NULL,
    acc_active boolean NOT NULL DEFAULT true,
    acc_last_login timestamp with time zone,
    CONSTRAINT vb_accounts_pkey PRIMARY KEY (acc_id),
    CONSTRAINT vb_accounts_acc_name_key UNIQUE (acc_name),
    CONSTRAINT vb_accounts_acc_code_key UNIQUE (acc_code)
)
WITH (
    OIDS = FALSE
)
TABLESPACE pg_default;

ALTER TABLE public.vb_accounts
    OWNER to gini;
'''

createTransactionsQry = '''
CREATE SEQUENCE IF NOT EXISTS public.vb_transactions_tx_id_seq
    INCREMENT 1
    START 1
    MINVALUE 1
    MAXVALUE 2147483647
    CACHE 1;

ALTER SEQUENCE public.vb_transactions_tx_id_seq
    OWNER TO gini;
    
CREATE TABLE IF NOT EXISTS public.vb_transactions
(
    tx_id integer NOT NULL DEFAULT nextval('public.vb_transactions_tx_id_seq'::regclass),
    tx_ctime timestamp with time zone NOT NULL DEFAULT now(),
    tx_ref character varying(10) COLLATE pg_catalog."default" NOT NULL,
    tx_account integer NOT NULL,
    tx_type character varying(40) COLLATE pg_catalog."default" NOT NULL,
    tx_amount double precision NOT NULL DEFAULT 0.0,
    tx_narrative text COLLATE pg_catalog."default" NOT NULL,
    CONSTRAINT vb_transactions_pkey PRIMARY KEY (tx_id),
    CONSTRAINT vb_transactions_tx_ref_key UNIQUE (tx_ref),
    CONSTRAINT tx_account FOREIGN KEY (tx_account)
        REFERENCES public.vb_accounts (acc_id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
)
WITH (
    OIDS = FALSE
)
TABLESPACE pg_default;

ALTER TABLE public.vb_transactions
    OWNER to gini;
'''

In [3]:
# connect to the database
def connectToDB(dbName='virtualbank', dbUser='youruser', dbPort=5432, dbPassword='yourpassword', dbHost='localhost'):
    # make the connection
    return psycopg2.connect(database=dbName, user=dbUser, host=dbHost, password=dbPassword, port=dbPort)
vbConn = connectToDB()
# get the cursor
vbCursor = vbConn.cursor()

In [4]:
# execute the create queries
vbCursor.execute(createAccountsQry)
vbCursor.execute(createTransactionsQry)

In [5]:
# commit changes
vbConn.commit()

<h2>Class Exercise</h2>

<p>Using the queries as in the SQLite session, create some accounts, read them from the database, update a record and delete a record.</p>